This notebook is intended to demonstrate how select registration, segmentation, and image mathematical methods of ITKTubeTK can be combined to perform lung segmentation from non-constrast CT.

There are many other (probably more effective) lung extraction methods available as open-source software.  If you need to perform lung extraction for a large collection of scans that do not contain major pathologies, please use one of those packages.   This notebook is meant to show off the capabilities of specific ITKTubeTK methods, not to demonstration how to "solve" lung extraction.

In [1]:
import itk
from itk import TubeTK as tube

from itkwidgets import view

import numpy as np
import time

In [2]:
ImageType = itk.Image[itk.F, 3]

InputBaseName = "../data/CT/CT"
OutputBaseName = "results/CT"

In [3]:
im = itk.imread(InputBaseName + ".mha", itk.F)

In [4]:
resamp = tube.ResampleImage[ImageType].New(Input = im)
resamp.SetMakeIsotropic(True)
resamp.Update()
imIso = resamp.GetOutput()

itkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'itkImageF4'
vectoritkImageF4 not loaded from module TubeTK because of exception:
 module 'itk.TubeTKPython' has no attribute 'vectoritkImageF4'


In [5]:
itk.imwrite(imIso, OutputBaseName + "-Iso.mha")

In [6]:
ImageMath = tube.ImageMath.New(imIso)
ImageMath.ConnectedComponents(-1024,-900,1,0,0,0)
ImageMath.Dilate(10,1,0)
ImageMath.Erode(20,1,0)
ImageMath.Dilate(11,1,0)
imBkgMask = ImageMath.GetOutput()

In [7]:
itk.imwrite(imBkgMask, OutputBaseName+'-BkgMask.mha')

In [8]:
ImageMath.SetInput(imIso)
ImageMath.ReplaceValuesOutsideMaskRange(imBkgMask,0,0,-1024)
ImageMath.Threshold(-1023,-500,1,0)
ImageMath.Dilate(5,1,0)
ImageMath.Erode(10,1,0)
ImageMath.Dilate(3,1,0)
imLungPlusMask = ImageMath.GetOutputUChar()

ConnComp = tube.SegmentConnectedComponents.New(imLungPlusMask)
ConnComp.SetMinimumVolume(10000)
ConnComp.Update()
imLungMask = ConnComp.GetOutput()

In [9]:
itk.imwrite(imLungMask, OutputBaseName+'-LungsMask.mha')

In [10]:
imLungMask = itk.imread(OutputBaseName+'-LungsMask.mha', itk.F) # Read into float array
ImageMath.SetInput(imIso)
ImageMath.ReplaceValuesOutsideMaskRange(imLungMask,1,255,-1025)
imLung = ImageMath.GetOutput()

In [11]:
itk.imwrite(imLung, OutputBaseName+'-Lungs.mha')

In [12]:
view(imLung)

<IPython.core.display.Javascript object>